# import basic 

In [ ]:
import numpy as np
import pandas as pd 

# load Data

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-oct-2021/test.csv")

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

In [ ]:
train.head(10)

# delete id column

In [ ]:
train.drop('id',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

# check target data

In [ ]:
train["target"].unique()

# install pycaret 

In [ ]:
 !pip install pycaret

# load pycaret

In [ ]:
import pycaret
from pycaret.classification import *

# make pycaret model

In [ ]:
classification  = setup(data=train, 
             target = 'target',                # Use your target variable.
             session_id=123, 
             log_experiment=True, 
             experiment_name='experiment1',     # Use any experiment name.
             silent=True ,                      # Runs the command without user input. 
#              remove_outliers=True,
             normalize=True,
             transformation=True,
             use_gpu = True
#              pca = True,
#              pca_components=200
            )

# compare models


In [ ]:
best = compare_models(sort = 'AUC',n_select=2, fold = 5,include = ['lightgbm','xgboost','catboost'])

# blend models

In [ ]:
print('Blending Models....')
blended = blend_models(estimator_list= best,fold=5, optimize='AUC')

# predict models

In [ ]:
pred_holdout = predict_model(blended)

# finalize model 

In [ ]:
final_model = finalize_model(blended)

# predict test data

In [ ]:
print('Done...!!!')
pred_esb = predict_model(final_model, test)
predictLabel = pred_esb['Label']

# submission

In [ ]:
output = pd.read_csv("../input/tabular-playground-series-oct-2021/sample_submission.csv")
output["target"] = predictLabel
output.to_csv('automl_submission.csv', index=False)